In [11]:
# Dependencies
from census import Census
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import wbdata


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")





In [12]:
# Importing data from Stats Canada
csv_url = "Resources/owid-covid-data.csv"
covid_data = pd.read_csv(csv_url)
covid_data.columns


Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [13]:
# Filtering DataFrame by columns of interest
clean_covid_data = covid_data.loc[covid_data['iso_code'].str.contains('OWID_')==False, ['iso_code', 'location', 'date', 'new_cases',  'new_deaths',  'new_cases_per_million',  'new_deaths_per_million',
                                      'icu_patients',
                                      'icu_patients_per_million', 'hosp_patients_per_million', 'new_tests_per_thousand',
                                      'positive_rate', 'people_vaccinated_per_hundred',
                                      'people_fully_vaccinated_per_hundred', 'stringency_index',
                                      'population_density', 'median_age',
                                      'gdp_per_capita', 'extreme_poverty', 'human_development_index', 'population']]


# Creating a list of columns to sum and calculate mean

sum_columns_list = ['new_cases',  'new_deaths',
                    'new_cases_per_million',  'new_deaths_per_million']
mean_columns_list = ['icu_patients','icu_patients_per_million','hosp_patients_per_million']                              

# Converting NaN to 0
clean_covid_data = clean_covid_data.fillna(0)

# Display DF
clean_covid_data.head(10)


,iso_code,location,date,new_cases,new_deaths,new_cases_per_million,new_deaths_per_million,icu_patients,icu_patients_per_million,hosp_patients_per_million,...,positive_rate,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,stringency_index,population_density,median_age,gdp_per_capita,extreme_poverty,human_development_index,population
0,AFG,Afghanistan,2020-02-24,5.0,0.0,0.122,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
1,AFG,Afghanistan,2020-02-25,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
2,AFG,Afghanistan,2020-02-26,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
3,AFG,Afghanistan,2020-02-27,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
4,AFG,Afghanistan,2020-02-28,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
5,AFG,Afghanistan,2020-02-29,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
6,AFG,Afghanistan,2020-03-01,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.78,54.422,18.6,1803.987,0.0,0.511,41128772.0
7,AFG,Afghanistan,2020-03-02,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.78,54.422,18.6,1803.987,0.0,0.511,41128772.0
8,AFG,Afghanistan,2020-03-03,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.78,54.422,18.6,1803.987,0.0,0.511,41128772.0
9,AFG,Afghanistan,2020-03-04,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.78,54.422,18.6,1803.987,0.0,0.511,41128772.0


In [14]:

# Creating a new column for Month-Year
clean_covid_data.insert(3,"month_year",pd.to_datetime(clean_covid_data['date']).dt.to_period('M') )
clean_covid_data_no_date = clean_covid_data.drop('date',axis = 1)

# Display DF
clean_covid_data_no_date.head(10)

,iso_code,location,month_year,new_cases,new_deaths,new_cases_per_million,new_deaths_per_million,icu_patients,icu_patients_per_million,hosp_patients_per_million,...,positive_rate,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,stringency_index,population_density,median_age,gdp_per_capita,extreme_poverty,human_development_index,population
0,AFG,Afghanistan,2020-02,5.0,0.0,0.122,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
1,AFG,Afghanistan,2020-02,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
2,AFG,Afghanistan,2020-02,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
3,AFG,Afghanistan,2020-02,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
4,AFG,Afghanistan,2020-02,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
5,AFG,Afghanistan,2020-02,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.33,54.422,18.6,1803.987,0.0,0.511,41128772.0
6,AFG,Afghanistan,2020-03,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.78,54.422,18.6,1803.987,0.0,0.511,41128772.0
7,AFG,Afghanistan,2020-03,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.78,54.422,18.6,1803.987,0.0,0.511,41128772.0
8,AFG,Afghanistan,2020-03,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.78,54.422,18.6,1803.987,0.0,0.511,41128772.0
9,AFG,Afghanistan,2020-03,0.0,0.0,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.78,54.422,18.6,1803.987,0.0,0.511,41128772.0


In [52]:
iso_codes = clean_covid_data_no_date['iso_code'].unique()
iso_codes

array(['AFG', 'ALB', 'DZA', 'AND', 'AGO', 'AIA', 'ATG', 'ARG', 'ARM',
       'ABW', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR',
       'BEL', 'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BES', 'BIH', 'BWA',
       'BRA', 'VGB', 'BRN', 'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN',
       'CPV', 'CYM', 'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COG',
       'COK', 'CRI', 'CIV', 'HRV', 'CUB', 'CUW', 'CYP', 'CZE', 'COD',
       'DNK', 'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI',
       'EST', 'SWZ', 'ETH', 'FRO', 'FLK', 'FJI', 'FIN', 'FRA', 'PYF',
       'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GRD',
       'GUM', 'GTM', 'GGY', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HKG',
       'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'IMN', 'ISR',
       'ITA', 'JAM', 'JPN', 'JEY', 'JOR', 'KAZ', 'KEN', 'KIR', 'KWT',
       'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LIE', 'LTU',
       'LUX', 'MAC', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MHL',
       'MRT', 'MUS',

In [15]:
# Creating a DataFrame grouping by Month-Year and Country and adding the columns for sums and average

df_sum = clean_covid_data_no_date.groupby(['iso_code', 'month_year'])[
    sum_columns_list].sum()



df_sum = df_sum.rename(columns={"new_cases":"sum_new_cases",'new_deaths':'sum_new_deaths', 'new_cases_per_million': 'sum_new_cases_per_million'\
                        , 'new_deaths_per_million': 'sum_new_deaths_per_million' })
                      

df_mean = clean_covid_data_no_date.groupby(['iso_code', 'month_year'])[
    mean_columns_list].mean()


df_mean = df_mean.rename(columns={'icu_patients': 'average_icu_patients',
                    'icu_patients_per_million': 'average_icu_patients_per_million','hosp_patients_per_million':'average_hosp_patients_per_million'})


df_clean_data = pd.merge(df_sum, df_mean, on=['iso_code', 'month_year'])

df_clean_data



sum_new_cases  sum_new_deaths  sum_new_cases_per_million  \
iso_code month_year                                                             
ABW      2020-03              55.0             0.0                    516.629   
         2020-04              45.0             2.0                    422.698   
         2020-05               1.0             1.0                      9.393   
         2020-06               2.0             0.0                     18.787   
         2020-07              18.0             0.0                    169.079   
...                            ...             ...                        ...   
ZWE      2022-10             428.0             4.0                     26.225   
         2022-11            1271.0            14.0                     77.877   
         2022-12             817.0            17.0                     50.059   
         2023-01            1625.0            15.0                     99.568   
         2023-02             718.0             6.0                     43.994   

                     sum_new_deaths_per_million  average_icu_patients  \
iso_code month_year                                                     
ABW      2020-03                          0.000                   0.0   
         2020-04                         18.786                   0.0   
         2020-05                          9.393                   0.0   
         2020-06                          0.000                   0.0   
         2020-07                          0.000                   0.0   
...                                         ...                   ...   
ZWE      2022-10                          0.244                   0.0   
         2022-11                          0.858                   0.0   
         2022-12                          1.042                   0.0   
         2023-01                          0.919                   0.0   
         2023-02                          0.368                   0.0   

                     average_icu_patients_per_million  \
iso_code month_year                                     
ABW      2020-03                                  0.0   
         2020-04                                  0.0   
         2020-05                                  0.0   
         2020-06                                  0.0   
         2020-07                                  0.0   
...                                               ...   
ZWE      2022-10                                  0.0   
         2022-11                                  0.0   
         2022-12                                  0.0   
         2023-01                                  0.0   
         2023-02                                  0.0   

                     average_hosp_patients_per_million  
iso_code month_year                                     
ABW      2020-03                                   0.0  
         2020-04                                   0.0  
         2020-05                                   0.0  
         2020-06                                   0.0  
         2020-07                                   0.0  
...                                                ...  
ZWE      2022-10                                   0.0  
         2022-11                                   0.0  
         2022-12                                   0.0  
         2023-01                                   0.0  
         2023-02                                   0.0  

[8016 rows x 7 columns]

In [16]:
# Create a DataFrame for Static Columns
totals_list=['stringency_index','population_density', 'median_age',
                                      'gdp_per_capita', 'extreme_poverty', 'human_development_index', 'population']
df_totals = clean_covid_data_no_date.groupby(
    ['iso_code'])[totals_list].mean()

# Converting population to Int
df_totals['population'] = df_totals['population'].astype(int)

df_totals


,stringency_index,population_density,median_age,gdp_per_capita,extreme_poverty,human_development_index,population
iso_code,,,,,,,
ABW,39.699157,584.800,41.2,35973.781,0.0,0.000,106459
AFG,28.103287,54.422,18.6,1803.987,0.0,0.511,41128772
AGO,53.224757,23.890,16.8,5819.495,0.0,0.581,35588996
AIA,0.000000,0.000,0.0,0.000,0.0,0.000,15877
ALB,42.891342,104.871,38.0,11803.431,1.1,0.795,2842318
...,...,...,...,...,...,...,...
WSM,0.000000,69.413,22.0,6021.557,0.0,0.715,222390
YEM,26.711897,53.508,20.3,1479.147,18.8,0.470,33696612
ZAF,43.973941,46.754,27.3,12294.876,18.9,0.709,59893884


#John Will do this:



In [17]:
wbdata.getGdpData()

[{'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2021',
  'value': 1537.3360209254,
  'unit': '',
  'obs_status': '',
  'decimal': 1},
 {'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2020',
  'value': 1353.7691597287,
  'unit': '',
  'obs_status': '',
  'decimal': 1},
 {'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3code': 'AFE',
  'date': '2019',
  'value': 1500.22927438203,
  'unit': '',
  'obs_status': '',
  'decimal': 1},
 {'indicator': {'id': 'NY.GDP.PCAP.CD',
   'value': 'GDP per capita (current US$)'},
  'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
  'countryiso3co

Gus will do this

In [54]:
# Creating dataframe from World Bank API JSON
gdp = wbdata.getGdpData()
gdp = [entry for entry in gdp if int(entry['date']) > 2018]
gdp_df = pd.DataFrame.from_dict(gdp)
gdp_by_year = gdp_df[['countryiso3code', 'date', 'value']]
gdp_by_year = gdp_by_year.rename(columns={'countryiso3code': 'country'})

# Fillining NaN with 0
gdp_by_year = gdp_by_year.fillna(0)

# Using .pivot_table
gdp_by_year = gdp_by_year.pivot_table(
    values='value', index='country', columns='date', aggfunc='first')

gdp_by_year = gdp_by_year.reset_index()

gdp_by_year = gdp_by_year[gdp_by_year['country'].isin(iso_codes)]

gdp_2019 = gdp_by_year.loc[:,['country', '2019']]
gdp_2019 = gdp_2019.loc[gdp_2019['2019']>0]
# gdp_2019 = gdp_2019.drop(index=0)


gdp_head = gdp_2019.sort_values(by='2019').head()
gdp_tail = gdp_2019.sort_values(by='2019').tail()


## Make this into a function to calculate other years


gdp_tail


date,country,2019
53,CYM,89871.913139
141,LUX,112621.821337
28,BMU,116153.166122
135,LIE,167019.618093
146,MCO,199377.481832
